In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [2]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
import dagshub

# 1. Set up MLflow experiment
dagshub.init(repo_owner='ashar-22', repo_name='hw02ml', mlflow=True)
experiment_name = 'xgb_experiment'
mlflow.set_experiment(experiment_name)

train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
# test_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
# test_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')

train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
# test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

features = [col for col in train.columns if col not in ["TransactionID", "isFraud"]]
X = train[features]
y = train["isFraud"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Accessing as ashar-22

Initialized MLflow to track repo "ashar-22/hw02ml"

Repository ashar-22/hw02ml initialized!

# Data Cleaning

In [3]:
with mlflow.start_run(run_name="XGBoost_Cleaning"):
    threshold = 0.75
    
    missing_ratio = X_train.isnull().mean()
    cols_to_drop = missing_ratio[missing_ratio > threshold].index.tolist()
    X_train.drop(columns=cols_to_drop, inplace=True)
    X_valid.drop(columns=cols_to_drop, inplace=True)
    
    mlflow.log_metric("initial_missing_ratio", missing_ratio.mean())
    mlflow.log_param("drop_threshold", threshold)
    mlflow.log_param("num_cols_dropped", len(cols_to_drop))

🏃 View run XGBoost_Cleaning at: https://dagshub.com/ashar-22/hw02ml.mlflow/#/experiments/1/runs/4fb069c5179b43ada7db983d06a50ecd
🧪 View experiment at: https://dagshub.com/ashar-22/hw02ml.mlflow/#/experiments/1


# Feature Engineering

In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

numerical_cols = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()

with mlflow.start_run(run_name="XGBoost_Feature_Engineering"):
    def add_transaction_amt_log(X):
        X = X.copy()
        if "TransactionAmt" in X.columns:
            X["TransactionAmt_log"] = np.log1p(X["TransactionAmt"])
        return X

    log_transformer = FunctionTransformer(add_transaction_amt_log, validate=False)

    numerical_transformer = Pipeline([
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ])

    categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ])

    if "TransactionAmt" in numerical_cols:
        mlflow.log_param("new_features", "TransactionAmt_log")

🏃 View run XGBoost_Feature_Engineering at: https://dagshub.com/ashar-22/hw02ml.mlflow/#/experiments/1/runs/98337032190145c2b251abbb09af9d2f
🧪 View experiment at: https://dagshub.com/ashar-22/hw02ml.mlflow/#/experiments/1


# Feature Selection

In [5]:
import mlflow
import pandas as pd
import numpy as np

with mlflow.start_run(run_name="XGBoost_Feature_Selection"):
    k_best = 150
    selector = SelectKBest(score_func=f_classif, k=k_best)
    selector.fit(X_train[numerical_cols].fillna(0), y_train)
    selected_features = np.array(numerical_cols)[selector.get_support()].tolist()

    if "TransactionAmt" in selected_features:
        selected_features.append("TransactionAmt_log")
    
    mlflow.log_param("num_selected_features", len(selected_features))
    mlflow.log_param("selected_features", ", ".join(selected_features))

🏃 View run XGBoost_Feature_Selection at: https://dagshub.com/ashar-22/hw02ml.mlflow/#/experiments/1/runs/3d746d35714647778393e370b4ba045f
🧪 View experiment at: https://dagshub.com/ashar-22/hw02ml.mlflow/#/experiments/1


# Training

In [6]:
from xgboost import XGBClassifier

with mlflow.start_run(run_name="XGBoost_Model_Training"):
    preprocessor = Pipeline([
        ("log", log_transformer),
        ("column_transform", ColumnTransformer([
            ("num", numerical_transformer, selected_features),
            ("cat", categorical_transformer, categorical_cols)
        ]))
    ])

    xgb = XGBClassifier(
        use_label_encoder=False,
        eval_metric='auc',
        random_state=42
    )

    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", xgb)
    ])

    param_grid = {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [4, 6],
        'classifier__learning_rate': [0.05, 0.1]
    }

    grid_search = GridSearchCV(pipeline, param_grid, scoring='roc_auc', cv=3, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    y_train_pred = best_model.predict_proba(X_train)[:, 1]
    y_valid_pred = best_model.predict_proba(X_valid)[:, 1]

    train_auc = roc_auc_score(y_train, y_train_pred)
    valid_auc = roc_auc_score(y_valid, y_valid_pred)

    mlflow.log_metrics({"train_auc": train_auc, "valid_auc": valid_auc})
    mlflow.log_params(best_params)

    mlflow.sklearn.log_model(best_model, "fraud_pipeline")

    print(f"Train AUC: {train_auc:.4f}")
    print(f"Validation AUC: {valid_auc:.4f}")

2025/04/27 14:51:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Train AUC: 0.9350
Validation AUC: 0.9215
🏃 View run XGBoost_Model_Training at: https://dagshub.com/ashar-22/hw02ml.mlflow/#/experiments/1/runs/0730a5d576154a4792027d2e794ce888
🧪 View experiment at: https://dagshub.com/ashar-22/hw02ml.mlflow/#/experiments/1
